In [1]:
import os

In [2]:
%pwd

'd:\\software\\python_vs\\lone_approval_prediction\\research'

In [4]:
os.chdir("../")

### Entity

In [5]:
from pathlib import Path
from dataclasses import dataclass

@dataclass
class DataTransformationConfig:
    root_dir : Path
    data_file_path : Path

# Configuration manager

In [6]:
from lone_approval_prediction.constants import *
from lone_approval_prediction.utils.comman import create_directories, read_yaml

class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
        ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self,config) -> DataTransformationConfig:
        config = self.config["data_transformation"]
        
        create_directories([self.config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir = config["root_dir"],
            data_file_path = config["data_file_path"]
            
        )
        
        return data_transformation_config
        

## componets

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from lone_approval_prediction import logger

class DataTransformation:
    def __init__(self,config:DataTransformationConfig):
        self.config = config
        
    def read_data_file(self):
        self.data = pd.read_csv(self.config.data_file_path)
        
    def drop_unwanted_columns(self,unwanted_features:list):
        for feature in unwanted_features:
            if feature in self.data.columns:
                data.drop(feature,axis=1,inplace=True)
                logger.info(f"{feature} has been removed from the dataset")
            else:
                logger.info(f"{feature} does dont exists in this dataset")
    
    def preprocess_data(self):
        encoding =  OrdinalEncoder()
        categorical_features = [feature for feature in self.data.columns if data[feature].dtype == "O"]
        for feature in categorical_features:
            try:
                self.data[f"{feature}_encodded"] = encoding.fit_transform(self.data[feature])
                logger.info(f"{feature} has been encodded successfully with {feature}_encodded")
                self.data.drop(feature,axis=1,inplace=True)
                logger.info(f"{feature} has been deleted successfully")
            except Exception as e:
                raise e
            
        scaler = StandardScaler
        
        sclaled_data = scaler.fit_transform(self.data)
        
        
            
        
            
    
            
        
        